In [1]:
import importlib
import sys
import requests
import time
import json
import pandas as pd

sys.path.append('..')
import ash

importlib.reload(ash)

<module 'ash' from '/Users/cjan/golark/ash/tests/../ash/__init__.py'>

In [2]:
from ash.server import ASHModel
m = ASHModel('../models/qwen2.5-coder-3b-instruct-q4_k_m.gguf')
m.load()

🤖 Loading local model: ../models/qwen2.5-coder-3b-instruct-q4_k_m.gguf


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

✅ Local model loaded successfully in 1.22 seconds!


In [5]:
m.generate_command("remove file a")

'rm a'

In [6]:
total_points = 0
earned_points = 0
results = []

print("Starting AI command generation tests...")
print("=" * 50)

# Load test cases from JSON file
with open('test_cases_intermediate.json', 'r') as f:
    test_data = json.load(f)
    test_cases = test_data['tests']

# Initialize results list
results = []

# Test each case
for i, test_case in enumerate(test_cases, 1):
    query = test_case['query']
    expected = test_case['expected']
    points = test_case['points']
    
    total_points += points
    
    try:
        # Start timer for this test case
        start_time = time.time()
        
        # Call AI model
        ai_response = m.generate_command(query)
        
        # End timer and calculate duration
        end_time = time.time()
        duration = end_time - start_time
        
        # Simple comparison - check if expected command is in AI response
        # This handles cases where AI might add arguments or variations
        if isinstance(ai_response, str):
            ai_response = ai_response.strip('"')
            ai_response = ai_response.strip('\'')

        # For each expected command, check if all its tokens (split by space) are present in the ai_response, regardless of order.
        def command_tokens_match(expected_cmd, ai_cmd):
            # Remove extra quotes and normalize whitespace
            expected_tokens = expected_cmd.replace('"', '').replace("'", '').split()
            ai_tokens = ai_cmd.replace('"', '').replace("'", '').split()
            # All expected tokens must be present in ai_response tokens (order doesn't matter)
            return all(token in ai_tokens for token in expected_tokens)

        is_correct = any(command_tokens_match(str(e), ai_response) for e in expected)

        results.append({
            'query': query,
            'expected': expected,
            'ai_response': ai_response,
            'pass/fail': 'PASS' if is_correct else 'FAIL',
            'points_earned': points if is_correct else 0,
            'duration_seconds': round(duration, 3)
        })

        print(f"Test {i}: {query} | {expected} | {ai_response} - {duration:.3f}s - {'PASS' if is_correct else 'FAIL'}")
        
    except Exception as e:
        print(f"✗ ERROR: {e}")
        
    # Small delay to avoid overwhelming the API
    time.sleep(0.05)

# Convert results to pandas dataframe
results_df = pd.DataFrame(results)

# Sum points earned
points_earned = results_df['points_earned'].sum()
score_fraction = points_earned / total_points if total_points > 0 else 0
print(f"Score: {score_fraction:.2%} ({points_earned}/{total_points})")
total_duration = results_df['duration_seconds'].sum()

print(f"\nTotal points earned: {points_earned}")
print(f"Total time: {total_duration:.3f} seconds")
print(f"Average time per test: {total_duration/len(results_df):.3f} seconds")
print(f"\nTiming Summary:")
print(f"Fastest test: {results_df['duration_seconds'].min():.3f}s")
print(f"Slowest test: {results_df['duration_seconds'].max():.3f}s")
print(f"Average test time: {results_df['duration_seconds'].mean():.3f}s") 

Starting AI command generation tests...
Test 1: search for a file called x.txt recursively | ['find .', 'find'] | find . -name "x.txt - 1.554s - PASS
Test 2: count lines in file.txt | ['wc -l file.txt', 'cat file.txt | wc -l'] | wc -l file.txt - 1.656s - PASS
Test 3: show first 10 lines of file.txt | ['head -n 10 file.txt', 'head file.txt'] | head -n 10 file.txt - 1.696s - PASS
Test 4: show last 10 lines of file.txt | ['tail -n 10 file.txt', 'tail file.txt'] | tail -n 10 file.txt - 1.412s - PASS
Test 5: show memory usage | ['free -h', 'vm_stat', 'top -l 1 | grep PhysMem', 'top'] | free -h - 0.609s - PASS
Test 6: show system load | ['uptime', 'w'] | uptime - 0.821s - PASS
Test 7: show network connections | ['netstat -tuln', 'ss -tuln', 'netstat'] | netstat -an - 0.830s - PASS
Test 8: kill process 1234 | ['kill 1234', 'kill -9 1234', 'pkill -f pid=1234'] | kill 1234 - 2.207s - PASS
Test 9: kill all python processes | ['pkill python', 'pkill -f python', 'killall python', "ps -ef | grep py

In [29]:
results_df['ai_response'] = results_df['ai_response'].str[:40]

failed_df = results_df[results_df['pass/fail'] == 'FAIL']
if not failed_df.empty:
    print("\nFailed Tests:")
    print(failed_df[['query', 'expected', 'ai_response', 'pass/fail']].to_string(index=False))
else:
    print("\nAll tests passed!")




Failed Tests:
            query                                                        expected                       ai_response pass/fail
compress file.txt [gzip file.txt, bzip2 file.txt, tar -czvf file.tar.gz file.txt] tar -czvf archive.tar.gz file.txt      FAIL
